In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid", palette="colorblind")
sns.set(rc={"figure.figsize": (11.5, 8.5), "figure.dpi": 100})

from dataset import SCIData, SCICols
%load_ext autoreload
%autoreload 1
%aimport utils.data_profiling, dataset

In [7]:
sci = SCIData.load('data/sci_processed.h5')

## Manual NEWS

The NEWS specification prescribes the level of clinical response depending on the aggregate score the patient receives [RCP17, pp.53, 62]. 
 - Score 0: Low clinical risk, continue routine monitoring.
 - Score 1-4: Low clinical risk, ward-based response, RN assesment of patient.
 - Score of 3 in any single parameter: Low-medium risk, clinician review.
 - Score ≥ 5: Medium clinical risk, urgent response threshold, urgent clinician assessment.
 - Score ≥ 7: High clinical risk, emergency response threshold, urgent assessment, consider transfer to higher-dependency unit or ICU.



In [22]:
sci[['DiedDuringStay', 'DiedWithin30Days', 'Readmitted', 'CriticalCare']].any(axis=1)

0         False
1         False
2          True
3         False
4         False
          ...  
170828    False
170829    False
170830    False
170831    False
170832    False
Length: 170833, dtype: bool